Import necessary libraries

In [38]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Get that webpage and do some basic actions with it

In [39]:
# idea for web scraping: scrape audible best-selling books
# Put the link to Audible Best-Sellers page here no matter the category
URL = "https://www.audible.com/search?sort=popularity-rank&ref=a_search_l1_catBackAll&pf_rd_p=daf0f1c8-2865-4989-87fb-15115ba5a6d2&pf_rd_r=S4VEHTG7BAZHCPTD9YB7"
URL = "https://www.audible.com/search?crid=31E3J6JGY980O&i=na-audible-us&k=brandon+sanderson&keywords=brandon+sanderson&ref-override=a_search_t1_header_search&sort=pubdate-desc-rank&sprefix=brand%2Cna-audible-us%2C170&url=search-alias%3Dna-audible-us&ref=a_search_c1_sort_1&pf_rd_p=073d8370-97e5-4b7b-be04-aa06cf22d7dd&pf_rd_r=VRAHHY22TC5WB0JFHJFR"
URL = "https://www.audible.com/adblbestsellers?searchCategory=18573518011&ref=a_adblbests_l1_catRefs_10&pf_rd_p=2ea8d46b-3372-49db-8ad4-77416e49695f&pf_rd_r=Z5CN827PYCK5J0XNQ8SM"
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, "html.parser")

In [40]:
# many thanks to https://jovian.ai/pratulofficialthings/audible-business-and-careers-books-2022
book_list = soup.find_all('li', class_='bc-list-item productListItem')


def get_book_names(book_list):
    book_names = []
    for tag in book_list:
        a_tag_name = tag.h3.find_all('a', recursive=False)
        book_name = a_tag_name[0].text.strip()
        book_names.append(book_name)
    return book_names

def get_book_length(book_list):
    book_length = []
    for tag in book_list:
        try:
            len_tag = tag.find('li', class_='bc-list-item runtimeLabel')
            length_tag = len_tag.find('span')
            length = length_tag.text.strip()
            book_length.append(length)
        except AttributeError:
            book_length.append(None)
    # Remove "Length: " from every elem of output list if it's not NoneType
    book_length = [i[8:] if i is not None else i for i in book_length]
    return book_length

def get_author(book_contents):
    written_by=[]
    for tag in book_contents:
        author_tag= tag.find('li', class_='bc-list-item authorLabel')
        try:
            auth_tag = author_tag.find('a')
            author = auth_tag.text.strip()
            written_by.append(author)
        except AttributeError:
            written_by.append(None)
    return written_by

def get_description(book_list):
    description = []
    for tag in book_list:
        about_tag = tag.find('li', class_='bc-list-item subtitle')
        try:
            description_tag = about_tag.find('span').text.strip()
            description.append(description_tag)
        except AttributeError:
            description.append(None)
    return description

def get_language(book_list):
    language_list = []
    for tag in book_list:
        lang_tag = tag.find('li', class_='bc-list-item languageLabel')
        try:
            language_tag = lang_tag.find('span').text.split()
            # Access 'English' in ['Language:', 'English'] of language_tag
            language_itself = language_tag[1]
            language_list.append(language_itself)
        except AttributeError:
            language_list.append(None)
    return language_list

def get_rating(book_list):
    rating = []
    for tag in book_list:
        star_tag = tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find(
                'span', class_='bc-text bc-pub-offscreen').text.strip()
            rating.append(rating_tag)
        except AttributeError:
            rating.append(None)
    return rating

def get_nunber_of_ratings(book_list):
    nunber_of_ratings = []
    for tag in book_list:
        star_tag = tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find(
                'span', class_='bc-text bc-size-small bc-color-secondary').text.strip()
            nunber_of_ratings.append(rating_tag)
        except AttributeError:
            nunber_of_ratings.append(None)
    return nunber_of_ratings

def get_regular_price(book_list):
    regular_price = []
    for tag in book_list:
        buy_tag = tag.find(
            'p', class_='bc-text buybox-regular-price bc-spacing-none bc-spacing-top-none')
        try:
            price_tag = buy_tag.find_all(
                'span', class_='bc-text bc-size-base bc-color-base')
            price = price_tag[1].text.strip()
            regular_price.append(price)
        except AttributeError:
            regular_price.append(None)
    return regular_price

def get_member_price(book_list):
    pass

df = pd.DataFrame(
{
    "Book_Name": get_book_names(book_list),
    "Description": get_description(book_list),
    "Author": get_author(book_list),
    "Rating": get_rating(book_list),
    "Num_of_Ratings": get_nunber_of_ratings(book_list),
    "Regular_Price": get_regular_price(book_list),
    "Audio_Length": get_book_length(book_list),
    "Language": get_language(book_list)
})

df.head()

,Book_Name,Description,Author,Rating,Num_of_Ratings,Regular_Price,Audio_Length,Language
0,Extreme Ownership,How U.S. Navy SEALs Lead and Win,Jocko Willink,5 out of 5 stars,"70,908 ratings",$15.22,9 hrs and 33 mins,English
1,And There Was Light,Abraham Lincoln and the American Struggle,Jon Meacham,5 out of 5 stars,342 ratings,$38.50,17 hrs and 49 mins,English
2,Sapiens,A Brief History of Humankind,Yuval Noah Harari,4.5 out of 5 stars,"51,594 ratings",$36.50,15 hrs and 18 mins,English
3,South to America,A Journey Below the Mason-Dixon to Understand ...,Imani Perry,4.5 out of 5 stars,203 ratings,$44.49,16 hrs and 32 mins,English
4,A Promised Land,None,Barack Obama,5 out of 5 stars,"51,850 ratings",$29.94,29 hrs and 10 mins,English


In [41]:
def check_book_availability(book_name_to_check, df):
    # check if the book is present in df
    # if not, finish the program straightaway
    # otherwise, check whether it was ranked before
    # Availability of rating is sufficient to check whether you can buy it or not
    if book_name_to_check in df["Book_Name"].values:
        print(f"\nThe book with the title '{book_name_to_check}' is present in the catalogue")
        book_characteristics = df.loc[df['Book_Name'] == book_name_to_check]
        if book_characteristics["Num_of_Ratings"].values.all() == "Not rated yet":
            print("but unfortunately isn't available for buying yet")
        elif book_characteristics["Rating"].values.all() is None:
            print("but unfortunately isn't available for buying yet")
        else:
            print("and is available for buying")
    else:
        print(f"\nThe book with the title '{book_name_to_check}' isn't present in the catalogue")

check_book_availability("The Lost Metal", df)
check_book_availability("fjdslkf", df)
check_book_availability("Centers of Gravity", df)
check_book_availability("The Shepherd's Crown", df)


The book with the title 'The Lost Metal' isn't present in the catalogue

The book with the title 'fjdslkf' isn't present in the catalogue

The book with the title 'Centers of Gravity' isn't present in the catalogue

The book with the title 'The Shepherd's Crown' isn't present in the catalogue


Create connection to MySQL

In [42]:
# later put related to MySQL code and imports into a separate file
from sqlalchemy import create_engine
from env_vars import DB_USER, DB_PASSWORD

def establish_connection():
    DB_TO_WORK_WITH = "audible_books_db"
    DB_HOST = "localhost:3306"
    return create_engine(
        f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_TO_WORK_WITH}",
        pool_recycle=3600)

engine = establish_connection()

Load df to db

In [43]:
# If you load to db straightaway, you will get dublicate indexes and (maybe) values
# To solve this problem, read everything from your DB table into df
# and append your just scraped info to it with resetting the index
# Warning: may be slow with big amount of data
def append_to_DB(input_df):
    all_info_from_db_df = pd.read_sql("select * from books;", engine, index_col="id")
    # combine two dfs
    df_combined = pd.concat([all_info_from_db_df, input_df])
    # remove dublicates and reset index
    df_combined.drop_duplicates(subset=["Book_Name", "Author", "Regular_Price", "Audio_Length", "Language"],
                                ignore_index=True, inplace=True)
    # load all info to DB
    df_combined.to_sql(name="books", if_exists='replace', con=engine, index_label="id")

append_to_DB(df)

Read from your db

In [45]:
pd.read_sql("select * from books;", engine, index_col="id")

,Book_Name,Description,Author,Rating,Num_of_Ratings,Regular_Price,Audio_Length,Language
id,,,,,,,,
0,The Light We Carry,Overcoming in Uncertain Times,Michelle Obama,5 out of 5 stars,21 ratings,$35.00,9 hrs and 59 mins,English
1,"Friends, Lovers, and the Big Terrible Thing",A Memoir,Matthew Perry,4.5 out of 5 stars,"5,548 ratings",$25.51,8 hrs and 49 mins,English
2,The Lost Metal,A Mistborn Novel,Brandon Sanderson,5 out of 5 stars,80 ratings,$31.18,18 hrs and 46 mins,English
3,Fairy Tale,None,Stephen King,5 out of 5 stars,"37,056 ratings",$26.90,24 hrs and 6 mins,English
4,It Starts with Us,A Novel,Colleen Hoover,4.5 out of 5 stars,"15,009 ratings",$23.62,8 hrs and 41 mins,English
5,I'm Glad My Mom Died,None,Jennette McCurdy,5 out of 5 stars,"71,391 ratings",$13.99,6 hrs and 26 mins,English
6,Atomic Habits,An Easy & Proven Way to Build Good Habits & Br...,James Clear,5 out of 5 stars,"113,173 ratings",$16.72,5 hrs and 35 mins,English
7,Desert Star,None,Michael Connelly,5 out of 5 stars,"1,604 ratings",$30.79,9 hrs and 37 mins,English
8,Spare,None,Prince Harry The Duke of Sussex,None,Not rated yet,$50.40,Not Yet Known,English
